In [1]:
def get_metrics(input_file):
    print("input file: ", input_file)
    f = open(input_file,"r")
    fl = f.readlines()
    f.close()

    count = 1
    metrics=[]
    split_title = input_file.split("_")
#     print(split_title)
    metrics.extend([float(split_title[3]), split_title[4].split(".")[0]])
    for line in fl:
    #     print(line)
        if count >= 32 and count < 37:
            _, val_str = line.split(":") 
            if defocus.match(line):
                val=float(val_str.split(",")[0])
                metrics.append(val)
    #             print(val)
            if score.match(line):
                val=float(val_str)
                metrics.append(val)
    #             print(val)
            if thon_rings.match(line):
                val=float(val_str.split("Angstroms")[0])
                metrics.append(val)
    #             print(val)
        count+=1
    all_metrics[split_title[0]].append(metrics)

In [2]:
import sys
import re
import os
from collections import defaultdict
import glob
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline


defocus = re.compile(r"Estimated defocus values[ ]+: [-+]?[0-9]*\.?[0-9]+ , [-+]?[0-9]*\.?[0-9]+ Angstroms")
score = re.compile(r"Score[ ]+: [-+]?[0-9]*\.?[0-9]+")
thon_rings = re.compile(r"Thon rings with good fit up to[ ]+: [-+]?[0-9]*\.?[0-9]+ Angstroms")

all_metrics = defaultdict(list)
for f in glob.glob('*summary*.txt'):
    get_metrics(f)

In [3]:
all_metrics

defaultdict(list, {})

In [5]:
plt.rcParams.update({'font.size': 22})
for i,stack in enumerate(all_metrics.keys()):
    num = i+1
    stack_num = int(stack.split('stack')[-1])
    stack_metrics=np.array(all_metrics[stack])
    hard_threshold_mul = sorted(set(stack_metrics.T[0]))
    print(hard_threshold_mul)

    bfactors = sorted(set(stack_metrics.T[1]))
    print(bfactors)

    twoddefocus=[]
    twodscores=[]
    twodfit=[]
    scores=[]
    fit=[]
    for thres in hard_threshold_mul:
        vals=np.array(list(filter(lambda x: x[0] == float(thres), stack_metrics)),dtype='float32')
        order = vals[:,1].argsort()
    #     print(order)
    #     print(vals[order])
        twoddefocus.append(list(vals[order][:,2]))
        twodscores.append(list(vals[order][:,3]))
        twodfit.append(list(vals[order][:,4]))
        means=vals.mean(axis=0)
        scores.append(means[3])
        fit.append(means[4])
    #     print(vals[:,3])
    # print(twodscores)
    # print(twodfit)
#     print(twodscores)
    extent = [min(bfactors), max(bfactors), max(hard_threshold_mul), min(hard_threshold_mul)]

    fig, axes = plt.subplots(1, 3,figsize=(30,10))
#     fig.subplots_adjust(left=0.02, bottom=0.06, right=0.95, top=0.94, wspace=0.05)

    # fig, ax = plt.subplots()
    # divider = make_axes_locatable(ax)
    # cax = divider.append_axes('right', size='5%', pad=0.05)

    mat = axes[0].matshow(twoddefocus,extent=extent,aspect='auto')
    axes[0].set_title('defocus {}'.format(stack_num))
    fig.colorbar(mat,ax=axes[0])

    mat = axes[1].matshow(twodscores,extent=extent,aspect='auto')
    axes[1].set_title('score {}'.format(stack_num))
    fig.colorbar(mat,ax=axes[1])

    mat = axes[2].matshow(twodfit,extent=extent,aspect='auto')
    axes[2].set_title('fit {}'.format(stack_num))
    fig.colorbar(mat,ax=axes[2])

['1.0']
['0']
[]


IndexError: too many indices for array